### Import Statements

In [46]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

##### Taking User Input (In this case, 'Mobile Covers')

In [47]:
user_input = 'Mobile Covers'

##### Fetching the HTML using Selenium Driver and initialising a soup object

In [48]:
driver = webdriver.Chrome(r"C:\Users\ASD\Downloads\chromedriver_win32\chromedriver")

search_url = "https://www.daraz.pk/catalog/?spm=a2a0e.searchlistcategory.search.2.3eac4b8amQJ0zd&q={input}&_keyori=ss&from=suggest_normal&sugg=samsung%20m20_1_1"

driver.get(search_url.format(input=user_input))

html = driver.page_source

soup = BeautifulSoup(html)

##### Fetches Total Sellers by searching the div class c1DXz4
##### Fetches product names by div class c16H9d

In [49]:
##  Total Sellers
total_sellers = int((soup.find('div', {'class': 'c1DXz4'})).span.text.split()[0])
                                                    
# print("Total Sellers: ",total_sellers)

##  Name
all_names = soup.find_all('div', {'class': 'c16H9d'})
item_name_list = []

for name in all_names:
    item_name_list.append(name.a.text)

# print(item_name_list)

###### Checks whether the product was in Daraz Mall or not by div class c3vCyH

In [50]:
is_daraz_mall = soup.find_all('div', {'class': 'c3vCyH'})
is_daraz_mall_list = []

for daraz_mall in is_daraz_mall:
    if((daraz_mall.i) == None):
        is_daraz_mall_list.append("No")
    else:
        is_daraz_mall_list.append("Yes")

# print(is_daraz_mall_list)

##### Fetches the Original and Discounted Price by the div class c3lr34 & c3gUW0 respectively

In [51]:
##  Original Price
original_price = soup.find_all('div', {'class': 'c3lr34'})
original_price_list = []

for o_price in original_price:
    if(o_price.span != None):
        original_price_list.append(int((o_price.span.text[4:]).replace(',', '')))
    else:
        original_price_list.append("No Discount")
                    
# print(original_price_list)

##  Discounted Price
discounted_price = soup.find_all('div', {'class': 'c3gUW0'})
discounted_price_list = []
        
for d_price in discounted_price:
    discounted_price_list.append(int((d_price.span.text[4:]).replace(',', '')))                

# print(discounted_price_list)

##### Fetches the total reviews by div class c15YQ9 and span class c3XbGJ

In [52]:
reveiws = soup.find_all('div', {'class': 'c15YQ9'})
reveiws_list = []

for review in reveiws:
    r = (review.find('span', {'class': 'c3XbGJ'}))
    
    if(r == None):
        reveiws_list.append(0)
    else:  
        r = str(r.text)
        reveiws_list.append(int(r[1:(len(r))-1]))

# print(reveiws_list)

##### Fetches the Seller Country by div class c15YQ9 and span class c2i43-

In [53]:
seller_country = soup.find_all('div', {'class': 'c15YQ9'})
seller_country_list = []

for seller in seller_country:
    s = (seller.find('span', {'class': 'c2i43-'}))
    
    seller_country_list.append(s.text)

# print(seller_country_list)

##### Calculates the product's rating by div class c15YQ9 and the following item tags

In [54]:
ratings = soup.find_all('div', {'class': 'c15YQ9'})
ratings_list = []

star0 = 0.0
star1 = 0.0
star25 = 0.0
star50 = 0.0
star75 = 0.0

for r in ratings:   
    
    if (str(r.i).find("c3dn4k c3EEAg") > -1):
        star1 += 1

    if(r.i == None):
        ratings_list.append(0)
        continue
        
    for tag in r.i.next_siblings:        
        if (str(tag).find("c3dn4k c3EEAg") > -1):
            star1 += 1.0
            
        elif (str(tag).find("c3dn4k c1dtTC") > -1):
            star0 += 0.0
        
        elif (str(tag).find("c3dn4k c3An30") > -1):
            star50 += 0.5
        
        elif (str(tag).find('c3dn4k c1Zozd') > -1): 
            star25 += 0.25
        
        elif (str(tag).find('c3dn4k c1wCjy') > -1):
            star75 += 0.75
            
    total_sum = star1 + star25 + star50 + star75 
    ratings_list.append(total_sum)
    
    star0 = 0.0
    star1 = 0.0
    star25 = 0.0
    star50 = 0.0
    star75 = 0.0

# print(ratings_list)

##### Creating a Data Frame using all the previously created lists

In [55]:
data_frame = pd.DataFrame(list(zip(item_name_list, is_daraz_mall_list, original_price_list, discounted_price_list, ratings_list, reveiws_list, seller_country_list)), columns = ['Name', 'Daraz Mall?', 'Original Price', 'Discounted Price', 'Rating', 'Reviews', 'Seller Country'])

## Final Output pt. 1

In [56]:
print("Total Seller:", total_sellers,"\n","Avg Price:", data_frame['Discounted Price'].mean(),"\n","Avg Reviews:", data_frame['Reviews'].mean(),"\n","Avg Rating:", data_frame['Rating'].mean())

Total Seller: 1599866 
 Avg Price: 488.325 
 Avg Reviews: 51.55 
 Avg Rating: 4.2375


## Final Output pt. 2

In [57]:
data_frame.head()

,Name,Daraz Mall?,Original Price,Discounted Price,Rating,Reviews,Seller Country
0,Nillkin Case for iPhone 12 Pro Max Slide Camer...,Yes,2020,1699,4.75,24,China
1,Nillkin Case for Xiaomi Mi 10T And Mi 10T Pro ...,Yes,2020,1469,4.00,112,China
2,Cartoon Pop Socket for Mobiles,No,400,119,5.00,6,Pakistan
3,Customized Universal Mobile Back Cover,No,1000,534,4.00,79,Pakistan
4,Customized Universal Mobile Back Cover,No,1000,570,4.00,44,Pakistan
